In [1]:
import pandas as pd
import numpy as np
import os
from pandas.tseries.offsets import Hour
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
%%time

path = '../data/aggregated/'
files = [x for x in filter(lambda x: x.endswith('.csv'), os.listdir(path))]

regs_05 = np.ravel( [int(x) for x in  pd.read_csv('../data/regs_05.csv', nrows=1).columns ])

data = pd.DataFrame()

for fname in files:
    aggr = pd.read_csv(path+fname, sep=',', index_col='region').T
    aggr = aggr.get(regs_05)

    month = pd.to_datetime(fname.replace('.csv', ''))
    offset_hour = np.array( [Hour(offset.replace('H', '')) for offset in aggr.index])
    aggr.index = [month + offset for offset in offset_hour]
    data = pd.concat([data, aggr], join='inner')



Wall time: 29.8 s


In [3]:
data.head()

region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2009-01-01 00:00:00,40,90,27,39,182,308,333,413,413,184,...,4,0,0,46,16,11,0,46,7,6
2009-01-01 01:00:00,39,108,31,52,225,330,252,355,279,154,...,8,0,2,6,18,6,1,23,4,3
2009-01-01 02:00:00,31,84,26,67,213,313,243,340,224,124,...,11,0,3,1,1,1,0,9,0,1
2009-01-01 03:00:00,27,78,17,49,186,280,237,285,181,125,...,12,0,0,0,0,0,0,6,0,0
2009-01-01 04:00:00,16,41,7,37,103,220,215,244,208,97,...,15,0,1,0,1,1,0,2,0,0


In [4]:
dups = data.index.duplicated(keep='last') 

data = data.loc[~dups].sort_index()

In [5]:
%%time

h1 = pd.Timedelta('1 hour')    # Сдвиг на 1 час
h2 = pd.Timedelta('2 hour')    
week = pd.Timedelta('167 hour')    # Сдвиг на неделю минус один час

df = pd.DataFrame(columns=data.columns)

for t, t1 in zip(data.index, data.index[1:]):
    delta_t = pd.Timedelta(t1-t)
    if delta_t == h2:
        df.loc[t+h1] = data.loc[t - week]

data = pd.concat([data,df])
data.sort_index(inplace=True)

Wall time: 2.65 s


In [6]:
with open('aggregated_hours_3week_2009.01_2016.06.csv',"w") as fout:
    data.to_csv(fout, index_label='date')